# Demo: Local LLM

## Ollama and Langchain

### What is Ollama?
- A local LLM runtime to run models (Llama, Mistral, Qwen, Phi, etc.) on local machine with simple commands.
- Pull & run models:
  ```bash
    ollama serve
    ollama pull llama3.2
    ollama run llama3.2
    ``` 
- Exposes an HTTP API (default http://localhost:11434) for chat, embeddings, and model management.
- Great for privacy, offline work, reproducible demos, and avoiding cloud costs.
- **Website:** [ollama.com](https://ollama.com)

---
### What is Langchain?
- A Python framework to build LLM apps by composing blocks:
- Models (ChatModel, LLM), Embeddings, Document Loaders, TextSplitters, VectorStores, and Chains/Runnables.
- Providing unified interface for different providers (local or cloud), plus utilities for RAG, tools, and orchestration.
- Building blocks to turn that into apps
- **Docs:** [LangChain Python Docs](https://docs.langchain.com/oss/python/langchain/overview)

## Set Up Environemnt

Download the necessary packages for building RAG pipelines

- langchain
Core framework for building LLM apps (chains, prompts, runnables). You use it for text splitters, message types, and composing the RAG flow.

- langchain_community
Community-maintained integrations that were split out of langchain. Includes loaders (e.g., PyPDFLoader) and many third-party connectors you call in your code.

- langchain-ollama
LangChain’s native driver for Ollama. Gives you ChatOllama (talk to local LLMs like llama3.2) and OllamaEmbeddings (create embeddings locally).

- ollama
Python client SDK for the Ollama server API.

After installing
- In notebooks, restart the kernel so new packages are picked up.


In [1]:
!pip install -U langchain langchain_community langchain-ollama ollama


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Configure base URL for Ollama

In [2]:
import os, warnings
warnings.filterwarnings("ignore")

# Change if your Ollama runs elsewhere
OLLAMA_BASE = os.getenv("OLLAMA_BASE", "http://localhost:11434")
print("Ollama base URL:", OLLAMA_BASE)

Ollama base URL: http://localhost:11434


### Interact with Ollama in Local

In [3]:
from langchain_ollama import ChatOllama
import os

llm = ChatOllama(
    model="llama3.2",   # or another local chat-capable model you've pulled
    base_url=OLLAMA_BASE,
)
 
reply = llm.invoke("Say hello in one short sentence.").content
print("Model reply:", reply)

Model reply: Hello!


In [4]:
import os
from langchain_ollama import ChatOllama

# Config
OLLAMA_BASE = os.getenv("OLLAMA_BASE", "http://localhost:11434")
MODEL = "llama3.2"
PROMPT = "Why the sky is blue?"

# LLM (temperature=0 for reproducibility)
llm = ChatOllama(model=MODEL, base_url=OLLAMA_BASE, temperature=0.0)
print("Ready:", MODEL, "@", OLLAMA_BASE)
print("----------------------------------------","\n")
print("User: ", PROMPT)
response = llm.invoke(PROMPT)
print("System: ", response.content)

Ready: llama3.2 @ http://localhost:11434
---------------------------------------- 

User:  Why the sky is blue?
System:  The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.

Here's what happens:

1. **Sunlight enters Earth's atmosphere**: When sunlight enters our atmosphere, it consists of a spectrum of colors, including all the colors of the visible light spectrum (red, orange, yellow, green, blue, indigo, and violet).
2. **Light interacts with tiny molecules**: The sunlight encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2) in the atmosphere.
3. **Scattering occurs**: These tiny molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths. This is known as Rayleigh scattering.
4. **Blue light is scattered in all directions**: As a result of this scattering, the blue light is distri

### Calculating token usage in Ollama

1. Estimating Input Tokens:
- Rule of Thumb: A common approximation for English text is that 1 token is roughly equal to 4 characters or ¾ of a word. So,input tokens can be estimated by counting characters and dividing by 4, or counting words and multiplying by ¾.

2. Tracking Output Tokens and Performance:
- Ollama API Response: When interacting with the Ollama API, the response payload (especially in non-streaming scenarios or at the end of a stream) will include fields like ```eval_count```(number of output tokens) and ```eval_duration``` (time taken to generate them).
- Verbose Output: Run Ollama with the ```--verbose``` flag to see token counts and timing information after each message during execution.

3. Measuring Tokens per Second:
- Calculate tokens per second by dividing the ```eval_count``` (number of output tokens) by the ```eval_duration``` (time taken to generate them, typically in nanoseconds) from the Ollama API response. This gives you a measure of the model's generation speed.

In [5]:
# 1. Estimating Input Tokens:
# Rule of Thumb:

def approx_tokens_chars(text: str) -> int:
    # ~4 chars ≈ 1 token (very rough, English-centric)
    return max(1, round(len(text) / 4))

def approx_tokens_words(text: str) -> int:
    # ~0.75 tokens per word (very rough)
    return max(1, round(len(text.split()) * 0.75))

print("PROMPT:", PROMPT)
print("chars:", len(PROMPT), "| words:", len(PROMPT.split()))
print("≈ tokens (chars/4):", approx_tokens_chars(PROMPT))
print("≈ tokens (words*0.75):", approx_tokens_words(PROMPT))

PROMPT: Why the sky is blue?
chars: 20 | words: 5
≈ tokens (chars/4): 5
≈ tokens (words*0.75): 4


In [8]:
# 2. Track output tokens & timings from LangChain → Ollama (non-streaming)

print("Reply:\n", response.content)

meta = getattr(response, "response_metadata", {}) or {}
prompt_tokens = meta.get("prompt_eval_count", 0)   # tokens in the input (as actually evaluated)
output_tokens = meta.get("eval_count", 0)          # tokens generated
eval_duration_ns = meta.get("eval_duration", 0)    # generation time (ns)
total_duration_ns = meta.get("total_duration", 0)  # end-to-end time (ns)

print("\n--- Metadata ---")
print("prompt_eval_count:", prompt_tokens)
print("eval_count:", output_tokens)
print("eval_duration (ns):", eval_duration_ns)
print("total_duration (ns):", total_duration_ns)

# 3. Compute tokens/sec (throughput) from metadata
def tokens_per_second(eval_count: int, duration_ns: int) -> float:
    return 0.0 if not duration_ns else eval_count / (duration_ns / 1e9)

# Use the LangChain metadata
print("Tokens/sec (generation only):",
      f"{tokens_per_second(output_tokens, eval_duration_ns):.2f}")

print("Tokens/sec (overall E2E):",
      f"{tokens_per_second(output_tokens, total_duration_ns):.2f}")

Reply:
 The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.

Here's what happens:

1. **Sunlight enters Earth's atmosphere**: When sunlight enters our atmosphere, it consists of a spectrum of colors, including all the colors of the visible light spectrum (red, orange, yellow, green, blue, indigo, and violet).
2. **Light interacts with tiny molecules**: The sunlight encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2) in the atmosphere.
3. **Scattering occurs**: These tiny molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths. This is known as Rayleigh scattering.
4. **Blue light is scattered in all directions**: As a result of this scattering, the blue light is distributed throughout the atmosphere, reaching our eyes from all parts of the sky.
5. **Our eyes perceive the blue col

In [9]:
# 4. Streaming counts (final chunk has totals)
chunks = list(llm.stream(PROMPT))
final = chunks[-1]
meta = getattr(final, "response_metadata", {}) or {}

print("Prompt tokens (stream):", meta.get("prompt_eval_count", 0))
print("Output tokens (stream):", meta.get("eval_count", 0))
print("eval_duration (ns):", meta.get("eval_duration", 0))
print("total_duration (ns):", meta.get("total_duration", 0))


Prompt tokens (stream): 31
Output tokens (stream): 347
eval_duration (ns): 4451618100
total_duration (ns): 9618820400


In [10]:
# 5. CLI verbose (quick inspection), run in terminal:
# ```ollama run llama3.2 --verbose```


CLI verbose (quick inspection), run in terminal:
> ```ollama run llama3.2 --verbose```

---
Example Output:

```
>>> Why the sky is blue?
The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.

Here's what happens:

1. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2).
2. These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths.
3. This is because the smaller molecules are more effective at scattering the shorter wavelengths.
4. As a result, the blue light is scattered in all directions and reaches our eyes from all parts of the sky.
5. Our eyes perceive this scattered blue light as the color of the sky.

The reason we don't see the sky as red or violet (the colors that are scattered more than blue) is because our atmosphere scatters these longer wavelengths even less than it does the shorter blue wavelengths. This is why the sky
typically appears blue during the day, especially in the direction of the sun.

It's worth noting that the color of the sky can change under different conditions:

* During sunrise and sunset, the light has to travel through more of the atmosphere, which scatters the shorter blue wavelengths even more, making the sky appear more red.
* At high altitudes or during intense thunderstorms, the scattered light can be filtered by more particles in the air, changing the apparent color of the sky.
* In areas with a lot of dust or pollution, the color of the sky can be altered due to additional scattering.

So, that's why the sky is blue!

total duration:       4.5327914s
load duration:        38.6557ms
prompt eval count:    31 token(s)
prompt eval duration: 44.3904ms
prompt eval rate:     698.35 tokens/s
eval count:           330 token(s)
eval duration:        4.4487313s
eval rate:            74.18 tokens/s
```

### Parameters Tuning

- temperature
  - Lower temperature → safer, more predictable.
  - Higher temperature → more creative/varied (and sometimes off-topic).
- seed
- top_p
  - Smaller top_p → model samples from a smaller “nucleus” of probable tokens → safer, simpler language.
  - Larger top_p → more diverse choices, slightly more creative output.
- top_k
  - Very small top_k may reduce vocabulary diversity; moderate values often fine.
- Max output length(num_predict)
  - Larger num_predict permits longer outputs (until the model naturally stops).

In [15]:
from langchain_ollama import ChatOllama
import os

# Tune parameters of LLM in Ollama
llm_poem = ChatOllama(
    model="llama3.2",
)
 
PROMPT = "Write a short (3–5 lines) poem about a cat that loves the moon."
poem_demo = llm_poem.invoke(PROMPT).content
print(poem_demo)

Silver light, her heart's delight
She watches moonbeams dance through night
Her eyes aglow, like stars above
A feline dreamer, under lunar love
She purrs and sleeps, in gentle moonlight.


In [17]:
import os, pandas as pd
from langchain_ollama import ChatOllama

# Config
OLLAMA_BASE = os.getenv("OLLAMA_BASE", "http://localhost:11434")
MODEL = "llama3.2"
PROMPT = "Write a short (3–5 lines) poem about a cat that loves the moon."

def run(prompt: str, **opts):
    """Invoke once with given options; return text + a few metrics for comparison."""
    llm = ChatOllama(model=MODEL, base_url=OLLAMA_BASE, **opts)
    msg = llm.invoke(prompt)
    meta = getattr(msg, "response_metadata", {}) or {}
    text = (msg.content or "").strip()
    # tokens/sec based on generation time
    eval_ns = meta.get("eval_duration", 0) or 0
    tps = (meta.get("eval_count", 0) / (eval_ns / 1e9)) if eval_ns else None
    return text, {
        "prompt_tokens": meta.get("prompt_eval_count", 0),
        "out_tokens": meta.get("eval_count", 0),
        "tps_gen": None if tps is None else round(tps, 2),
    }

def preview(text: str, n: int = 140) -> str:
    s = (text or "").replace("\n", " ⏎ ")
    return s[:n] + ("…" if len(s) > n else "")

print("Ready:", MODEL, "@", OLLAMA_BASE)


Ready: llama3.2 @ http://localhost:11434


In [24]:
# Different experiments
EXPS = [
    {"label": "baseline",              "temperature": 0.7, "top_p": 0.9,  "num_predict": 256,  "seed": 42},
    {"label": "temp=0.0",              "temperature": 0.2, "top_p": 0.9,  "num_predict": 256, "seed": 42},
    {"label": "temp=1.2",              "temperature": 1.2, "top_p": 0.9,  "num_predict": 256, "seed": 42},
    {"label": "seed=80",              "temperature": 0.7, "top_p": 0.9,  "num_predict": 256, "seed": 80},
    {"label": "top_p=0.6",             "temperature": 0.7, "top_p": 0.6,  "num_predict": 256, "seed": 42},
    {"label": "top_p=0.98",            "temperature": 0.7, "top_p": 0.98, "num_predict": 256, "seed": 42},
    {"label": "top_k=20",              "temperature": 0.7, "top_p": 0.9,  "top_k": 20,        "num_predict": 256, "seed": 42},
    {"label": "num_predict=47",        "temperature": 0.7, "top_p": 0.9,  "num_predict": 47,  "seed": 42},
]

rows = []
for exp in EXPS:
    opts = {k: v for k, v in exp.items() if k != "label"}
    text, m = run(PROMPT, **opts)
    rows.append({
        "label": exp["label"],
        "temperature": opts.get("temperature"),
        "top_p": opts.get("top_p"),
        "top_k": opts.get("top_k"),
        "repeat_penalty": opts.get("repeat_penalty"),
        "num_predict": opts.get("num_predict"),
        "seed": opts.get("seed"),
        "out_tokens": m["out_tokens"],
        "tps_gen": m["tps_gen"],
        "Poem": text,
    })

df = pd.DataFrame(rows).set_index("label")

from IPython.display import display

styled = (
    df.style
      .set_properties(subset=['Poem'], **{
          'white-space': 'pre-wrap',   
          'font-family': 'monospace',  
      })
      .set_table_styles([
          {'selector': 'th', 'props': [('text-align', 'left')]},
          {'selector': 'td', 'props': [('vertical-align', 'top')]}
      ])
)

display(styled)


,temperature,top_p,top_k,repeat_penalty,num_predict,seed,out_tokens,tps_gen,Poem
label,,,,,,,,,
baseline,0.700000,0.900000,nan,None,256,42,46,57.560000,"Silver light upon her back, The moon's soft glow, her favorite fact. She purrs with joy, and eyes aglow, Her love for night, forever to show. In lunar beams, she finds her place."
temp=0.0,0.200000,0.900000,nan,None,256,42,49,84.730000,"Under lunar light, she purrs with glee, A midnight stalker, wild and free. Her eyes shine bright like stars above, As she dances to the moon's soft love. In its gentle beams, her heart finds home."
temp=1.2,1.200000,0.900000,nan,None,256,42,47,81.970000,"Softly creeps the feline form, Drawn to moonbeams on the storm, Her eyes aglow with lunar light, A midnight dancer, lost in sight. In moonlit nights, she's free to roam."
seed=80,0.700000,0.900000,nan,None,256,80,47,83.830000,"Silver light, you beckon me, A midnight feline, wild and free. I prowl beneath your gentle glow, My whiskers twitching, my heart aglow. In lunar dreams, I am meant to be."
top_p=0.6,0.700000,0.600000,nan,None,256,42,44,83.780000,"Silver light upon her back, The moon's soft glow, she loves to track. In midnight skies, she dances free, A feline queen, beneath the sea. Her heart beats bright, with lunar pace."
top_p=0.98,0.700000,0.980000,nan,None,256,42,45,82.150000,"Silver light upon her back, The moon's gentle beams, she'll gladly track. With eyes aglow like shining stone, She purrs and prowls beneath its throne. In lunar love, she finds her home."
top_k=20,0.700000,0.900000,20.000000,None,256,42,46,80.530000,"Silver light upon her back, The moon's soft glow, her favorite fact. She purrs with joy, and eyes aglow, Her love for night, forever to show. In lunar beams, she finds her place."
num_predict=47,0.700000,0.900000,nan,None,47,42,46,82.070000,"Silver light upon her back, The moon's soft glow, her favorite fact. She purrs with joy, and eyes aglow, Her love for night, forever to show. In lunar beams, she finds her place."


### Prompting 

There're a few prompt styles: zero-shot, few-shot, role + constraints, and structured JSON.

In [25]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2", base_url=OLLAMA_BASE, temperature=0.3)

prompt = """You are a helpful assistant.
Explain the concept of Artificial Intelligence in 3 bullet points, plain language."""
print(llm.invoke(prompt).content)

Here are three simple explanations of Artificial Intelligence (AI) in plain language:

• **AI is like a super-smart computer**: AI uses special algorithms and data to learn from information, make decisions, and solve problems on its own. It's designed to perform tasks that would be difficult or time-consuming for humans.

• **AI can learn and improve over time**: Unlike traditional computers, which follow pre-programmed instructions, AI systems can adjust their behavior based on new information and experiences. This means they can get better at what they do with each use, kind of like how we learn from our mistakes!

• **AI is used in many areas, not just tech**: While AI is often associated with technology, it's actually being applied to all sorts of fields, such as healthcare, finance, transportation, and even education. It can help analyze data, recognize patterns, and make predictions – making life easier and more efficient for people!


In [33]:
from langchain_core.messages import SystemMessage, HumanMessage

msgs = [
    SystemMessage(content="You are a precise technical writer. Use short, formal sentences."),
    HumanMessage(content="Explain AI to a 5-year-old in a playful tone."),
]
print(llm.invoke(msgs).content)


Imagine you have a super smart robot friend! 

This robot is called Artificial Intelligence, or AI for short.

It's like a computer that can think and learn, just like YOU!

But instead of using toys and blocks, it uses special math and codes to understand the world.

AI can help us with lots of things, like:

* Playing games with us
* Recognizing pictures and faces
* Even helping doctors make people feel better

It's like having a magic helper that makes our lives easier!

But remember, AI is still just a machine, so we need to be kind and teach it nice things too!


In [29]:
few_shot_prompt = """
You are a concise assistant. Answer with one sentence.

Q: What is tokenization in NLP?
A: Converting text into smaller units (tokens) for processing.

Q: What is a vector embedding?
A: A numeric representation of text that captures semantic meaning.

Q: What is Artificial Intelligence?
A:
""".strip()

print(llm.invoke(few_shot_prompt).content)


Artificial Intelligence refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.


In [30]:
role_prompt = """
## SYSTEM
You are a technical writing assistant. Be precise and avoid fluff.

## TASK
Summarize the definition of Artificial Intelligence.

## REQUIREMENTS
- 3 bullet points
- Each bullet ≤ 15 words
- No marketing language
""".strip()

print(llm.invoke(role_prompt).content)


* Artificial Intelligence (AI) refers to the simulation of human intelligence in machines.
* AI systems use algorithms and data to perform tasks that typically require human intelligence.
* The goal of AI is to create machines that can think and learn like humans.


In [32]:
# Structured JSON output

import json, re

json_prompt = """
Return a JSON object with keys: "title", "summary", and "keywords".
Topic: "Definition of Artificial Intelligence"
Keep summary < 40 words. keywords = array of 3–5 short items.
""".strip()

raw = llm.invoke(json_prompt).content
print("Raw output:\n", raw)


Raw output:
 Here is the JSON object:

```
{
    "title": "Definition of Artificial Intelligence",
    "summary": "Artificial intelligence (AI) refers to the simulation of human intelligence in machines, enabling them to perform tasks that typically require human intelligence.",
    "keywords": ["Machine Learning", "Deep Learning", "Natural Language Processing"]
}
```


In [34]:
draft = llm.invoke("Define AI in 2 sentences, plain language.").content

critique_prompt = f"""
You will refine the draft to meet ALL constraints:
- ≤ 35 words total
- No marketing language
- Avoid buzzwords

DRAFT:
{draft}

Return the FINAL version only.
""".strip()

print("DRAFT:\n", draft, "\n")
print("FINAL:\n", llm.invoke(critique_prompt).content)


DRAFT:
 Artificial Intelligence (AI) refers to computer systems that can think and learn like humans, using algorithms and data to make decisions and perform tasks on their own. These systems can be used for a wide range of applications, from simple automation to complex problem-solving, and are becoming increasingly integrated into many aspects of modern life. 

FINAL:
 Computer systems that think and learn like humans use algorithms and data to make decisions and perform tasks on their own. They're used in various applications, including automation and problem-solving.


In [47]:
# Example: Math problem solving with different prompt styles + self-consistency

import os, re, collections, pandas as pd
from langchain_ollama import ChatOllama

# Config
OLLAMA_BASE = os.getenv("OLLAMA_BASE", "http://localhost:11434")
MODEL = "llama3.2"
QUESTION = "What is the sum of the first 10 positive integers?"

# LLMs (tweak temps if you like)
llm        = ChatOllama(model=MODEL, base_url=OLLAMA_BASE, temperature=0.2)
llm_brief  = ChatOllama(model=MODEL, base_url=OLLAMA_BASE, temperature=0.3)
llm_plan   = ChatOllama(model=MODEL, base_url=OLLAMA_BASE, temperature=0.4)

# Prompts
prompt_answer_only = f"""
Solve the problem and output ONLY the final integer answer.
No units. No steps. No explanation.

Problem: {QUESTION}
""".strip()

# Brief rationale
prompt_brief = f"""
Solve the problem. Provide:
1) Brief rationale in ≤ 15 words
2) Final answer on a new line as: ANSWER: <number>

Problem: {QUESTION}
""".strip()

# Plan
prompt_plan = f"""
Create a very short 3-step plan (each ≤ 5 words) to solve:

{QUESTION}

Then on a new line, output final answer as:
ANSWER: <number>

Do not show intermediate calculations.
""".strip()

# Self-consistency helper
def answer_only_sample(q, seed=None, temp=0.8):
    llm_sc = ChatOllama(model=MODEL, base_url=OLLAMA_BASE, temperature=temp, seed=seed)
    m = llm_sc.invoke(f"Solve and output ONLY the final integer answer.\n\nProblem: {q}")
    txt = (m.content or "").strip()
    nums = re.findall(r"-?\d+", txt)
    return nums[0] if nums else txt

# Gather outputs
ans_only = llm.invoke(prompt_answer_only).content.strip()
brief    = llm_brief.invoke(prompt_brief).content.strip()
plan     = llm_plan.invoke(prompt_plan).content.strip()

top_counts = counter.most_common()
top_counts_str = ", ".join(f"{ans}×{cnt}" for ans, cnt in top_counts)
sc_note = "(Self-consistency: 7 samples, temp=0.8, seeds=1..7)"
sc_prompt_text = prompt_answer_only + "\n\n" + sc_note


# Self-consistency majority vote
answers  = [answer_only_sample(QUESTION, seed=s) for s in [1,2,3,4,5,6,7]]
counter  = collections.Counter(answers)
majority = counter.most_common(1)[0][0]

# Build table
rows = [
    {"method": "Answer-only", "prompt": prompt_answer_only, "output": ans_only},
    {"method": "Brief rationale", "prompt": prompt_brief, "output": brief},
    {"method": "Plan → Answer", "prompt": prompt_plan, "output": plan},
    {"method": "Self-Consistency (majority)", "prompt": sc_prompt_text, "output": majority},
]

df_prompt = pd.DataFrame(rows)

from IPython.display import display

styled_prompt_table = (
    df_prompt.style
      .set_properties(subset=['output'], **{
          'white-space': 'pre-wrap', 
          'font-family': 'monospace', 
      })
      .set_table_styles([
          {'selector': 'th', 'props': [('text-align', 'left')]},
          {'selector': 'td', 'props': [('vertical-align', 'top')]},
      ])
)
display(styled_prompt_table)

print("Self-Consistency vote distribution:", top_counts_str)


,method,prompt,output
0,Answer-only,Solve the problem and output ONLY the final integer answer. No units. No steps. No explanation. Problem: What is the sum of the first 10 positive integers?,55
1,Brief rationale,Solve the problem. Provide: 1) Brief rationale in ≤ 15 words 2) Final answer on a new line as: ANSWER: Problem: What is the sum of the first 10 positive integers?,"Rationale: Using the formula for the sum of an arithmetic series, S = n(n+1)/2. ANSWER: 55"
2,Plan → Answer,"Create a very short 3-step plan (each ≤ 5 words) to solve: What is the sum of the first 10 positive integers? Then on a new line, output final answer as: ANSWER: Do not show intermediate calculations.",Step 1: List first 10 numbers. Step 2: Add them together quickly. Step 3: Get total sum result. ANSWER: 55
3,Self-Consistency (majority),"Solve the problem and output ONLY the final integer answer. No units. No steps. No explanation. Problem: What is the sum of the first 10 positive integers? (Self-consistency: 7 samples, temp=0.8, seeds=1..7)",55


Self-Consistency vote distribution: 55×4, 10×2, 1×1


---
### Build Simple Chatbot Interface:

- Streamlit
- Gradio
- Flask

In [66]:
!pip install streamlit gradio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chainlit 1.3.2 requires packaging<24.0,>=23.1, but you have packaging 24.2 which is incompatible.
chainlit 1.3.2 requires python-multipart<0.0.10,>=0.0.9, but you have python-multipart 0.0.20 which is incompatible.
crawl4ai 0.5.0.post6 requires aiofiles>=24.1.0, but you have aiofiles 23.2.1 which is incompatible.
graphrag 0.0.1.dev49 requires aiofiles<25.0.0,>=24.1.0, but you have aiofiles 23.2.1 which is incompatible.
graphrag 0.0.1.dev49 requires nltk==3.8.1, but you have nltk 3.9.1 which is incompatible.
graphrag 0.0.1.dev49 requires tenacity<9.0.0,>=8.5.0, but you have tenacity 9.0.0 which is incompatible.
graphrag 0.0.1.dev49 requires tiktoken<0.8.0,>=0.7.0, but you have tiktoken 0.11.0 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pi

  Attempting uninstall: aiofiles
    Found existing installation: aiofiles 24.1.0
    Uninstalling aiofiles-24.1.0:
      Successfully uninstalled aiofiles-24.1.0


In [67]:
import os, gradio as gr
from langchain_ollama import ChatOllama

OLLAMA_BASE = os.getenv("OLLAMA_BASE", "http://localhost:11434")
llm = ChatOllama(model="llama3.2", base_url=OLLAMA_BASE, temperature=0.2)

def reply(message, history): 
    out = llm.invoke(message).content
    return out

demo = gr.ChatInterface(
    fn=reply,
    title="Simple Chatbot (Gradio + Ollama)",
    description="Type a message and get a response from a local model.",
)
demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
